In [1]:
#!/usr/bin/env python
#import SciServer.LoginPortal as Login
#token=Login.getToken()
#user = Login.getKeystoneUserWithToken(token)
#import SciServer.CasJobs as CasJobs # query with CasJobs
#import SciServer.SciDrive           # read/write to/from SciDrive
import urllib
from bs4 import BeautifulSoup as bs
import pandas
import numpy as np
import os
import datetime
import dateutil
#print(token)
print("Done")

Done


In [7]:
# First, get the list of dead celebrity names from the CSV file in SciDrive
#def scidrivePublicURL(path):
#    req = urllib.request.Request(url=SciServer.Config.SciDriveHost+'/vospace-2.0/1/media/sandbox/'+path,method='GET') 
#    req.add_header('X-Auth-Token', token) 
#    req.add_header('Content-Type','application/xml') 
#    res=urllib.request.urlopen(req)
#    jsonResponse = json.loads(res.read().decode())
#    return jsonResponse['url']
#container = 'fame'
#filename = container+"/death1732.csv"

# Load the list of names, first into a pandas dataframe, then into an Nx1 numpy array.
#rawdata = SciServer.SciDrive.download(filename,token=token)
#deadpandas = pandas.read_csv('deathunicode2.csv',encoding='utf-8')
deadpandas = pandas.read_csv('data/deathlist10.csv')
deadpandas['cause'] = deadpandas['cause'].replace(np.nan,'',regex=True)
deadpandas
#print(len(deadpandas))
deadpeople = deadpandas.values
deadpeople = deadpeople[:,1:]
#print(deadpeople)
#print(deadpeople.shape)
print(deadpeople[deadpeople[:,8] != 0])

[[ 'Garry Marshall, 81, American director, producer, writer, and actor (Happy Days, Pretty Woman, Murphy Brown), pneumonia.[305]'
  'https://en.wikipedia.org/wiki/Garry_Marshall' '2016-07-19 00:00:00'
  'Garry Marshall' 81.0 'American' 'pneumonia'
  ' director producer writer and actor (Happy Days Pretty Woman Murphy Brown)'
  2 34]
 [ "Mark Takai, 49, American politician, member of the U.S. House of Representatives for Hawaii's 1st district (since 2015) and the Hawaii House of Representatives (1994–2015), pancreatic cancer.[323]"
  'https://en.wikipedia.org/wiki/Mark_Takai' '2016-07-20 00:00:00'
  'Mark Takai' 49.0 'American' 'pancreatic cancer'
  " politician member of the U.S. House of Representatives for Hawaii's 1st district (since 2015) and the Hawaii House of Representatives (1994–2015)"
  1 48]
 [ 'Dennis Green, 67, American football coach (Minnesota Vikings, Arizona Cardinals, Northwestern Wildcats), heart attack.[333]'
  'https://en.wikipedia.org/wiki/Dennis_Green' '2016-07-2

In [8]:
# The article_lengths array (also Nx1 numpy) will hold length of each Wikip article in chars
article_lengths = np.zeros([len(deadpeople),1],dtype=np.int) 

for i in range(0,len(deadpeople)): # iterate through each name
#    print(i)
    # Construct the Wikip URL and read its contents
#    name = str(deadpeople[i,0])
#    print(name)
#    name2 = urllib.parse.quote(name.replace(" ","_"))
#    print(name2)
#    url = "http://en.wikipedia.org/wiki/"+name2
    url = deadpeople[i,1]
#    print(i,":",url)
    page = urllib.request.urlopen(url).read()
    # Parse HTML to find the contents of every <p>...</p> tag
    soup = bs(page, "html.parser")
#    soup.prettify()
#    heading1 = soup.find('h1')
    x = soup.find_all('p')
    
    # Calculate the length of all the text within the <p>...</p> tags
    lent = 0
    for j in x:
        lent = lent + len(j.get_text())
    article_lengths[[i][0]] = lent
#    if  (i > 500):
#        print(i,":",url)
#        print(article_lengths[i][0])

    # instant gratification: print the loop variable every 50 turns so I see it's doing something
    if (i % 50 == 0):
        print(i)
    #   print(article_lengths[i][0])

# Add article length to list of names
deadpeople = np.hstack((deadpeople,article_lengths))

print('Done!')
print(deadpeople)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
Done!
[['Les Stocker, 73, British wildlife expert, founder of Tiggywinkles.[295]'
  'https://en.wikipedia.org/wiki/Les_Stocker' '2016-07-18 00:00:00' ..., 0
  0 3010]
 [ 'Abu Wardah, 39, Indonesian militant leader (Mujahidin Indonesia Timur), shot by police.[296]'
  'https://en.wikipedia.org/wiki/Abu_Wardah' '2016-07-18 00:00:00' ..., 0 0
  4066]
 ['Bommi Baumann, 68, German author and political activist.[297]'
  'https://en.wikipedia.org/wiki/Bommi_Baumann' '2016-07-19 00:00:00' ...,
  0 0 3351]
 ..., 
 [ 'Charmian Carr, 73, American actress and singer (The Sound of Music), complications from dementia.[257]'
  'https://en.wikipedia.org/wiki/Charmian_Carr' '2016-09-17 00:00:00' ...,
  0 0 3306]
 ['Desmond Clarke, 74, Irish philosopher.[258]'
  'https://en.wikipedia.org/wiki/Desmond_Clarke' '2016-09-17 00:00:00' ...,
  0 0 940]
 [ 'Bahman Golbarnezhad, 48, Iranian Paralympic racing cyclist (2012, 2016), race co

In [9]:
# Results will eventually be in "deadpeople" - come back to it at the end

# Store the number of revisions & editors in a new array called "revisionarray"
revisionarray = np.zeros([len(deadpeople),2],dtype=int)

#revisionarray = np.zeros([1,2],dtype=int)
#for i in range(0,1):

# Iterate through all the names in deadpeople
for i in range(0,len(deadpeople)):
    name = str(deadpeople[i,3])
#    name = "David Bowie"
#    name = "Agha Nasir"
#    name = "Marvin Minsky"
#    name2 = urllib.parse.quote(name.replace(" ","_"))
    print(name)
    
    # Wikipedia displays revisions max 500 at a time. 
    iteration = 0
    completed = 0
    earliestrev = ''
    rownumber = 0
    
    while (completed == 0):
        iteration = iteration + 1

        mainurl = deadpeople[i,1].split('/')
        historyurl = ''
        for k in range(0,3):
            historyurl += str(mainurl[k])+'/'
        historyurl += 'w/index.php?title='
        historyurl += str(mainurl[4])        
        historyurl += "&offset="
        historyurl += earliestrev
        historyurl += "&limit=500&action=history"
        
        if (deadpeople[i,3] == 'Mike Hart'):
            historyurl = 'https://en.wikipedia.org/w/index.php?title=Mike_Hart_(singer/songwriter)&action=history'
#        print(historyurl)
        page = urllib.request.urlopen(historyurl).read()
        soup = bs(page, "html.parser")

        histolist = soup.find('ul',id='pagehistory')
        revisions = histolist.find_all('li')
        
        if (iteration == 1):
            revisor = np.empty([len(revisions),4],dtype='object')
        else:
            moreroom = np.empty(([len(revisions),4]),dtype='object')
            revisor = np.vstack((revisor,moreroom))

        if (len(revisions) < 500):
            completed = 1
        for j in revisions:
            if j.find(class_='history-deleted'):
                revisor[rownumber] = [dateutil.parser.parse('9999-12-31 23:59'),'','','']
            else:
                revisor[rownumber,0] = dateutil.parser.parse(j.find(class_='mw-changeslist-date').get_text())
                revisor[rownumber,1] = j.find(class_='mw-userlink').get_text()
                byte_change_string = ''
                byte_change_string = j.find(class_='history-size').get_text()        
                byte_change_string = byte_change_string.replace('(','')
                byte_change_string = byte_change_string.replace(')','')
                if (byte_change_string.find('bytes') > -1):
                    byte_change_string = byte_change_string.replace(' bytes','')
                elif (byte_change_string.find('byte') > -1):
                    byte_change_string = byte_change_string.replace(' byte','')                    
                byte_change_string = byte_change_string.replace(',','')
                if (byte_change_string == 'empty'):
                    revisor[rownumber,2] = 0
                else:
                    revisor[rownumber,2] = int(byte_change_string)
                plusorminusy = ''
                plusorminusy = j.find(class_='mw-plusminus-pos')
                if (plusorminusy == None):
                    plusorminusy = j.find(class_='mw-plusminus-neg')
                if (plusorminusy == None):
                    plusorminusy = j.find(class_='mw-plusminus-null')
                character_change_string = plusorminusy.get_text()
                character_change_string = character_change_string.replace('(','')
                character_change_string = character_change_string.replace(')','')
                character_change_string = character_change_string.replace('+','')
                character_change_string = character_change_string.replace(',','')
                revisor[rownumber,2] = int(character_change_string)
            rownumber = rownumber + 1
        earliestrev =  datetime.datetime.strftime(np.min(revisor[:,0]),'%Y%m%d%H%M%S')
    revisionarray[i,0] = (revisor.shape[0])
    revisionarray[i,1] = np.unique(revisor[:,[1]]).size
    print(i," : ",revisionarray[i])
deadpeople = np.hstack((deadpeople,revisionarray))
#print(deadpeople)
print("Done!")

Les Stocker
0  :  [6 6]
Abu Wardah
1  :  [57 25]
Bommi Baumann
2  :  [58 39]
Ray Bell
3  :  [6 5]
Betsy Bloomingdale
4  :  [155  71]
Chief Zee
5  :  [150  92]
Dimitri
6  :  [101  55]
Carlos Gorostiza
7  :  [57 32]
Carmen Hernández
8  :  [34 24]
Nev Hewitt
9  :  [21  9]
Garry Marshall
10  :  [1098  493]
Tom McCready
11  :  [19 14]
Gordon Mowrer
12  :  [13  7]
John Pidgeon
13  :  [67 31]
Anthony D. Smith
14  :  [140  93]
Tamás Somló
15  :  [12 10]
Dominique Arnaud
16  :  [26 19]
Radu Beligan
17  :  [141  88]
Dick Corballis
18  :  [5 4]
William Gaines
19  :  [59 40]
André Isoir
20  :  [57 39]
James Allen Johnson
21  :  [32  7]
Walid Juffali
22  :  [127  28]
György Kéri
23  :  [27 21]
Egon Matijevic
24  :  [14  9]
Ray Moreton
25  :  [2 1]
Jim Pressdee
26  :  [28 21]
Mohammed Shahid
27  :  [140  77]
Pavel Sheremet
28  :  [150  65]
Mark Takai
29  :  [208 101]
Adolph Bachmeier
30  :  [51 34]
Milford Burriss
31  :  [7 5]
Bill Cardille
32  :  [141  83]
Chor Yeok Eng
33  :  [37 19]
Tsering Chung

In [10]:
print('top')
linkarray = np.zeros([len(deadpeople),1],dtype=int)
#linkarray = np.zeros((1,1),dtype=int)
for i in range(0,len(deadpeople)):
#    print(i,'.',deadpeople[i,3])
    inbound_link_url = ''
    completed = 0
    nextpagelink = ''
    iteration = 0
    rownumber = 0
    while (completed == 0):
        iteration = iteration + 1
        if (nextpagelink == ''):
            mainurl = str(deadpeople[i,1]).split('/')
            for k in range(0,3):
                inbound_link_url += str(mainurl[k])+'/'
            inbound_link_url += 'w/index.php?title=Special:WhatLinksHere/'
            inbound_link_url += str(mainurl[4])
            inbound_link_url += "&limit=500"
        else:
            inbound_link_url = nextpagelink
        if (deadpeople[i,3] == 'Mike Hart'):
            inbound_link_url = 'https://en.wikipedia.org/wiki/Special:WhatLinksHere/Mike_Hart_(singer/songwriter)'
        try:
            page = urllib.request.urlopen(inbound_link_url).read()
        except:
            print('failed to parse on i=',i)
            print(i,':',inbound_link_url)

        soup = bs(page, "html.parser")
        nextpage_element = soup.find('a',string="next 500")
        if (nextpage_element != None):
            nextpagelink = "https://en.wikipedia.org/" + nextpage_element.get('href')
        else:
            completed = 1

        listoflinks = soup.find('ul',id='mw-whatlinkshere-list')

        inlinks = listoflinks.find_all('li')
              
        if (iteration == 1):
            linktracker = np.empty([len(inlinks),1],dtype=object)    
#            print('linktracker initialized to length ',len(linktracker))
        else:
#            print("size of linktracker before = ",linktracker.shape)
            moreroom = np.empty([len(inlinks),1],dtype=object)
#            print("size of moreroom = ",moreroom.shape)
            linktracker = np.vstack((linktracker,moreroom))
#            print("size of linktracker = ",linktracker.shape)
        
        for j in inlinks:
#            print(name)
#            print(rownumber)
#            print(j)
            linktracker[rownumber,0] = (bs.find(j,'a').get('href'))
            rownumber = rownumber + 1
        
#        print(linktracker)
    linkarray[i,0] = (linktracker.shape[0])
    print(i,':',deadpeople[i,3],':',linkarray[i,0])
#    print(linkarray[i,0])

deadpeople = np.hstack((deadpeople,linkarray))
print(deadpeople)

top
0 : Les Stocker : 4
1 : Abu Wardah : 19
2 : Bommi Baumann : 21
3 : Ray Bell : 6
4 : Betsy Bloomingdale : 34
5 : Chief Zee : 199
6 : Dimitri : 20
7 : Carlos Gorostiza : 34
8 : Carmen Hernández : 12
9 : Nev Hewitt : 36
10 : Garry Marshall : 500
11 : Tom McCready : 7
12 : Gordon Mowrer : 3
13 : John Pidgeon : 17
14 : Anthony D. Smith : 70
15 : Tamás Somló : 21
16 : Dominique Arnaud : 44
17 : Radu Beligan : 47
18 : Dick Corballis : 8
19 : William Gaines : 17
20 : André Isoir : 40
21 : James Allen Johnson : 8
22 : Walid Juffali : 22
23 : György Kéri : 10
24 : Egon Matijevic : 4
25 : Ray Moreton : 6
26 : Jim Pressdee : 18
27 : Mohammed Shahid : 48
28 : Pavel Sheremet : 151
29 : Mark Takai : 104
30 : Adolph Bachmeier : 18
31 : Milford Burriss : 4
32 : Bill Cardille : 26
33 : Chor Yeok Eng : 12
34 : Tsering Chungtak : 16
35 : Dick Donnelly : 6
36 : Bernard Dufour : 18
37 : John Garton : 24
38 : Roger Godement : 49
39 : Dennis Green : 480
40 : Joy Hardon : 17
41 : William F. Hickey Jr. : 5


In [11]:
#deadpeople = deadpeople[:,1:]
#print(deadpeople.shape)

x = pandas.DataFrame(deadpeople,columns=['text','url','death_date','name','age','nationality','cause','description','votes','sortorder','article_length','versions','editors','inbound_links'])
#x.set_index('id')
x.index.name = 'id'
#x
outfile = str(os.getcwd())+'/output/death10x.csv'
x.to_csv(outfile,encoding="utf-8")

print('Done!')

Done!
